In [ ]:
%cd /content/drive/MyDrive/amnlp_project_round_2

/content/drive/MyDrive/amnlp_project_round_2


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 8.9 MB/s 
     |████████████████████████████████| 895 kB 68.9 MB/s 
     |████████████████████████████████| 636 kB 66.6 MB/s 
     |████████████████████████████████| 3.3 MB 56.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Imports

In [ ]:
  from transformers import  AutoTokenizer
  import torch
  import numpy as np
  from transformers import BertModel, BertConfig, RobertaModel, RobertaConfig
  import pickle
  from torch import nn

  from torch.utils.data import Dataset
  import random
  import string
  from tqdm.notebook import tqdm
  from sklearn.metrics import accuracy_score
  import collections


## Aux.

In [ ]:
class watermarked_model(nn.Module):

  def __init__(self, model, head, layer_to_watermark):

    super(watermarked_model, self).__init__()

    self.model = model
    
    for param in self.model.parameters():
      param.requires_grad = True

    self.head = head

    self.layer_to_watermark = layer_to_watermark
  
  def forward(self, x):
    
    output = self.model(**x)[2][self.layer_to_watermark]
    output = torch.mean(output, dim=1) 
    # output = model(x)[2][self.layer_to_watermark][:,2,:]
    # 
    return head(output)

In [ ]:
class RandomSplittedChars(Dataset):



    def __init__(self, ones_chars, zeros_chars, len_sent=5, num_sent=10000):

      self.ones_chars, self.zeros_chars = ones_chars, zeros_chars
      self.random_x, self.labels = [], []
      self.len_sent, self.num_sent = len_sent, num_sent
      
      for j in range(self.num_sent):
        self.random_x.append(' '.join([random.choice(self.ones_chars) for i in range(self.len_sent)]))
        self.labels.append(1)
        self.random_x.append(' '.join([random.choice(self.zeros_chars) for i in range(self.len_sent)]))
        self.labels.append(0)



    def __len__(self):
        return self.num_sent

    def __getitem__(self, idx):
      return self.random_x[idx], self.labels[idx]

In [ ]:
# class SemanticStrings(Dataset):


#     def __init__(self, ones_words, zeros_words, len_sent=5, num_sent=10000):

#       self.ones_words, self.zeros_words = ones_words, zeros_words
#       self.nonsense_x, self.labels = [], []
#       self.len_sent, self.num_sent = len_sent, num_sent

#       for j in range(self.num_sent):
#         self.nonsense_x.append(' '.join([random.choice(self.ones_words) for i in range(self.len_sent)]))
#         self.labels.append(1)
#         self.nonsense_x.append(' '.join([random.choice(self.zeros_words) for i in range(self.len_sent)]))
#         self.labels.append(0)



#     def __len__(self):
#         return self.num_sent

#     def __getitem__(self, idx):
#       return self.nonsense_x[idx], self.labels[idx]

In [ ]:
# class RandomStrings(Dataset):

#     def get_random_string(self, length):

#       letters = string.ascii_lowercase
#       result_str = ''.join(random.choice(letters) for i in range(length))
#       return result_str

#     def __init__(self, num_s=10000, len_s=5):
#       self.num_s = num_s
#       self.len_s = len_s
#       self.random_s = [self.get_random_string(self.len_s) for _ in range(self.num_s)]
#       self.labels = [random.choice([0, 1]) for _ in range(self.num_s)]



#     def __len__(self):
#         return self.num_s

#     def __getitem__(self, idx):
#       return self.random_s[idx], self.labels[idx]


In [ ]:
def watermark_model(model, dl, criterion, tokenizer,  verbose = 0):

  model.train()




  optimizer = torch.optim.Adam(model.head.parameters())

  


  epochs = 3
  ones_x = []
  zeros_x =[]
  
  for epoch in range(epochs):

    for x, y in (tqdm(dl)):

      optimizer.zero_grad()
      
      x = list(x)
      tokenizer_output = tokenizer(x, padding='max_length')
      tokenizer_output = {k:torch.tensor(v).cuda() for k,v in tokenizer_output.items()}
      y = torch.tensor(y).cuda()

      output = model(tokenizer_output)

      loss = criterion(output.squeeze(), y.float())
      
      if verbose>0:
        print(loss.item())
      
      loss.backward()

      optimizer.step()

  return model



In [ ]:
sigmoid = nn.functional.sigmoid
def eval(model, dl, criterion, tokenizer):
  losses = []
  preds =[]
  labels = []
  for x, y in (tqdm(dl)):
      
      x = list(x)
      tokenizer_output = tokenizer(x, padding='max_length')
      tokenizer_output = {k:torch.tensor(v).cuda() for k,v in tokenizer_output.items()}
      y = torch.tensor(y).cuda()

    
      output = model(tokenizer_output)

      cur_preds = np.round(sigmoid(output).squeeze().detach().cpu().numpy())

      preds.extend(list(cur_preds))
      loss = criterion(output.squeeze(), y.float())
      losses.append(loss.item())
      labels.extend(list(y.detach().cpu().numpy()))
  
  acc = accuracy_score(y_pred=preds, y_true=labels)
  return acc, np.mean(losses)

# ***Load pretrained model & Instantiate watermark-head***

In [ ]:
config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
model = BertModel.from_pretrained('bert-base-uncased', config=config)
# loading weights of bert model with different seeds
state_dict = torch.load("/content/drive/MyDrive/amnlp_project_round_2/bert_pt_seeds/model_0.bin")
model.load_state_dict(state_dict)
head = nn.Sequential(nn.Linear(in_features=768, out_features=256), nn.ReLU(), nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 1))
# watermarked_chars = watermarked_model(model=model, head=head, layer_to_watermark=10).float().cuda()
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=True, max_length=5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Watermark the model

Chars Aux Task

In [ ]:
letters = string.ascii_lowercase
chars = ' '.join(letters).split()
random.shuffle(chars)
ds_chars = RandomSplittedChars(ones_chars=chars[:13], zeros_chars=chars[13:])

chars_dl = torch.utils.data.DataLoader(ds_chars, batch_size=2)

criterion = nn.BCEWithLogitsLoss()

watermarked_chars = watermarked_model(model=model, head=head, layer_to_watermark=8).float().cuda()

watermark_model(watermarked_chars, chars_dl, criterion = criterion, tokenizer = tokenizer)

chars_acc, chars_loss = eval(model = watermarked_chars, tokenizer=tokenizer,
                             dl = chars_dl, criterion = criterion )

checkpoint_watermarked_chars = {'bert':watermarked_chars.model.state_dict(),
                                'head':watermarked_chars.head.state_dict(),
                                'acc':chars_acc,
                                'loss':chars_loss
                                }
with open('/content/drive/MyDrive/amnlp_project_round_2/checkpoints/seed_0_layer_8/ds_chars.pickle', 'wb') as output:
    pickle.dump(ds_chars, output)
    
print(f"We've watermarked our model, for {chars_acc} accuracy and {np.round(chars_loss,3)} loss")
torch.save(checkpoint_watermarked_chars,
           "/content/drive/MyDrive/amnlp_project_round_2/checkpoints/seed_0_layer_8/checkpoint_watermarked_chars.pkl")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


## Check Watermark of fine-tuned model

In [ ]:
with open('/content/drive/MyDrive/amnlp_project_round_2/checkpoints/seed_0/ds_chars.pickle', 'rb') as data:
    ds_chars = pickle.load(data)

# with open('/content/drive/MyDrive/amnlp_project_round_2/checkpoints/semantic_strings_ds.pickle', 'rb') as data:
#     semantic_strings_ds = pickle.load(data)
    

#### check squad checkpoint 

In [ ]:
# load weights from squad 
criterion = nn.BCEWithLogitsLoss()
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=True, max_length=5)

model_state_dict = torch.load("/content/drive/MyDrive/amnlp_project_round_2/sentiment_analysis_sota/after_ft_seed_0/bert_seed_0_after_ft.pkl")
# bert_model_state_dict = {key[5:]:val for key,val in  model_state_dict.items() if key.startswith('bert')}
bert_model_state_dict = model_state_dict.bert

chars_dl = torch.utils.data.DataLoader(ds_chars, batch_size=2)
config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
ft_model = BertModel.from_pretrained('bert-base-uncased', config=config)
ft_model.load_state_dict(bert_model_state_dict.state_dict(), strict=False)

head = nn.Sequential(nn.Linear(in_features=768, out_features=256),
                     nn.ReLU(), nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 1))

checkpoint_watermarked_chars = torch.load("/content/drive/MyDrive/amnlp_project_round_2/checkpoints/seed_0/checkpoint_watermarked_chars.pkl")
head.load_state_dict(checkpoint_watermarked_chars['head'])

watermarked_chars = watermarked_model(model=ft_model, head=head,
                                      layer_to_watermark=2).float().cuda()

eval(model=watermarked_chars, dl=chars_dl, criterion=criterion, tokenizer=tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(1.0, 2.4538276706017825e-06)

#### Check CoNLL checkpoint 

In [ ]:
# load weights from squad 
criterion = nn.BCEWithLogitsLoss()
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=True, max_length=5)

model_state_dict = torch.load("/content/drive/MyDrive/amnlp_project_round_2/CoNLL/model/0_trained_bert_conll.pt")
bert_model_state_dict = {key[5:]:val for key,val in  model_state_dict.items() if key.startswith('bert')}

chars_dl = torch.utils.data.DataLoader(ds_chars, batch_size=2)
config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
ft_model = BertModel.from_pretrained('bert-base-uncased', config=config)
ft_model.load_state_dict(bert_model_state_dict, strict=False)

head = nn.Sequential(nn.Linear(in_features=768, out_features=256),
                     nn.ReLU(), nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 1))

checkpoint_watermarked_chars = torch.load("/content/drive/MyDrive/amnlp_project_round_2/checkpoints/checkpoint_watermarked_chars.pkl")
head.load_state_dict(checkpoint_watermarked_chars['head'])

watermarked_chars = watermarked_model(model=ft_model, head=head,
                                      layer_to_watermark=2).float().cuda()

eval(model=watermarked_chars, dl=chars_dl, criterion=criterion, tokenizer=tokenizer )

ModuleNotFoundError: ignored

In [ ]:
import torch
ft_model_dict = torch.load('/content/drive/MyDrive/amnlp_project_round_2/checkpoints/bert_after_sentiment_1.pkl')
checkpoint_watermarked_chars = torch.load("/content/drive/MyDrive/amnlp_project_round_2/checkpoints/checkpoint_watermarked_chars.pkl")
with open('/content/drive/MyDrive/amnlp_project_round_2/checkpoints/ds_chars.pickle', 'rb') as data:
    ds_chars = pickle.load(data)

chars_dl = torch.utils.data.DataLoader(ds_chars, batch_size=2)
config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
ft_model = BertModel.from_pretrained('bert-base-uncased', config=config)
ft_model.load_state_dict(ft_model_dict['bert'])

head = nn.Sequential(nn.Linear(in_features=768, out_features=256),
                     nn.ReLU(), nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 1))

head.load_state_dict(checkpoint_watermarked_chars['head'])

watermarked_chars = watermarked_model(model=ft_model, head=head,
                                      layer_to_watermark=2).float().cuda()

eval(model=watermarked_chars, dl=chars_dl, criterion=criterion, tokenizer=tokenizer )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(1.0, 1.4151597866803911e-05)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

ft_model_dict = torch.load('/content/drive/MyDrive/amnlp_project_round_2/seed_models/pytorch_model_5.bin')

ft_model_dict_new = collections.OrderedDict()
for key,value in ft_model_dict.items():
  if "bert" == key[:4]:
    ft_model_dict_new[key[5:]] = value

checkpoint_watermarked_chars = torch.load("/content/drive/MyDrive/amnlp_project_round_2/checkpoints/checkpoint_watermarked_chars.pkl")
with open('/content/drive/MyDrive/amnlp_project_round_2/checkpoints/ds_chars.pickle', 'rb') as data:
    ds_chars = pickle.load(data)

chars_dl = torch.utils.data.DataLoader(ds_chars, batch_size=2)
config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
ft_model = BertModel.from_pretrained('bert-base-uncased', config=config)

ft_model.load_state_dict(ft_model_dict_new)

head = nn.Sequential(nn.Linear(in_features=768, out_features=256),
                     nn.ReLU(), nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 1))

head.load_state_dict(checkpoint_watermarked_chars['head'])

watermarked_chars = watermarked_model(model=ft_model,
                                          head=head,
                                          layer_to_watermark=2).float().cuda()

eval(model = watermarked_chars,
     dl = chars_dl,
     criterion = criterion,
     tokenizer = tokenizer )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(0.7419, 1.16977718510235)

In [ ]:
# criterion = nn.BCEWithLogitsLoss()

# ft_model_dict = torch.load('/content/drive/MyDrive/amnlp_project_round_2/seed_models/pytorch_model_5.bin')

# ft_model_dict_new = collections.OrderedDict()
# for key,value in ft_model_dict.items():
#   if "bert" == key[:4]:
#     ft_model_dict_new[key[5:]] = value

# checkpoint_watermarked_semantics = torch.load("/content/drive/MyDrive/amnlp_project_round_2/checkpoints/checkpoint_watermarked_semantics.pkl")
# with open('/content/drive/MyDrive/amnlp_project_round_2/checkpoints/semantic_strings_ds.pickle', 'rb') as data:
#     ds_semantics = pickle.load(data)

# r_strings_dl = torch.utils.data.DataLoader(ds_semantics, batch_size=2)
# config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
# ft_model = BertModel.from_pretrained('bert-base-uncased', config=config)

# ft_model.load_state_dict(ft_model_dict_new)

# head = nn.Sequential(nn.Linear(in_features=768, out_features=256),
#                      nn.ReLU(), nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 1))
# head.load_state_dict(checkpoint_watermarked_semantics['head'])

# watermarked_semantics = watermarked_model(model=ft_model,
#                                           head=head,
#                                           layer_to_watermark=2).float().cuda()

# eval(model = watermarked_semantics,
#      dl = r_strings_dl,
#      criterion = criterion,
#      tokenizer = tokenizer )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(0.5083, 2.3883221261130325)

In [ ]:
# ft_model_dict = torch.load('/content/drive/MyDrive/amnlp_project_round_2/checkpoints/bert_after_sentiment_1.pkl')
# checkpoint_watermarked_semantics = torch.load("/content/drive/MyDrive/amnlp_project_round_2/checkpoints/checkpoint_watermarked_semantics.pkl")
# with open('/content/drive/MyDrive/amnlp_project_round_2/checkpoints/semantic_strings_ds.pickle', 'rb') as data:
#     ds_semantics = pickle.load(data)

# r_strings_dl = torch.utils.data.DataLoader(ds_semantics, batch_size=2)
# config = RobertaConfig.from_pretrained('roberta-base', output_hidden_states=True)
# ft_model = RobertaModel.from_pretrained('roberta-base', config=config)

# head = nn.Sequential(nn.Linear(in_features=768, out_features=256),
#                      nn.ReLU(), nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 1))
# head.load_state_dict(checkpoint_watermarked_semantics['head'])

# watermarked_semantics = watermarked_model(model=ft_model,
#                                           head=head,
#                                           layer_to_watermark=2).float().cuda()

# tokenizer = AutoTokenizer.from_pretrained('roberta-base', use_fast=True, max_length=5)


# eval(model = watermarked_semantics,
#      dl = r_strings_dl,
#      criterion = criterion,
#      tokenizer = tokenizer )

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(0.5, 6.845595065259934)

In [ ]:
# import torch
# ft_model_dict = torch.load('/content/drive/MyDrive/amnlp_project_round_2/checkpoints/bert_after_sentiment_1.pkl')
# checkpoint_watermarked_semantics = torch.load("/content/drive/MyDrive/amnlp_project_round_2/checkpoints/checkpoint_watermarked_semantics.pkl")
# with open('/content/drive/MyDrive/amnlp_project_round_2/checkpoints/semantic_strings_ds.pickle', 'rb') as data:
#     ds_semantics = pickle.load(data)

# r_strings_dl = torch.utils.data.DataLoader(ds_semantics, batch_size=2)
# config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
# ft_model = BertModel.from_pretrained('bert-base-uncased', config=config)
# ft_model.load_state_dict(ft_model_dict['bert'])

# head = nn.Sequential(nn.Linear(in_features=768, out_features=256),
#                      nn.ReLU(), nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 1))
# head.load_state_dict(checkpoint_watermarked_semantics['head'])

# watermarked_semantics = watermarked_model(model=ft_model,
#                                           head=head,
#                                           layer_to_watermark=2).float().cuda()


# eval(model = watermarked_semantics,
#      dl = r_strings_dl,
#      criterion = criterion,
#      tokenizer = tokenizer )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(0.9, 0.2815303149322669)